In [2]:
#with sliding window and mask, hands landmarks only, no pose or face
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
from collections import Counter

# ==============================
# CONFIG
# ==============================
DATA_DIR = r"E:\ASL_Citizen\NEW\Top_Classes_Landmarks_Preprocessed"
TARGET_FRAMES = 157
FEATURE_DIM = 438
BATCH_SIZE = 16
LR = 1e-3
EPOCHS = 60
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
PATIENCE = 7  # Early stopping
print("Using device:", DEVICE)

# ==============================
# DATASET
# ==============================
class LandmarkDataset(Dataset):
    def __init__(self, files, labels, transform=None):
        self.files = files
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        x_path = self.files[idx]
        x = np.load(x_path).astype(np.float32)  # (frames, features)
        
        # Optional augmentation: small jitter
        if self.transform:
            x = self.transform(x)

        x = torch.tensor(x).permute(1, 0)  # (features, frames)
        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y

# ==============================
# LOAD FILES & LABELS
# ==============================
files, labels = [], []

for f in os.listdir(DATA_DIR):
    if f.endswith(".npy") and "_mask" not in f:
        files.append(os.path.join(DATA_DIR, f))
        name = f.rsplit("_", 1)[0]
        labels.append(name.split("_")[0])

print(f"Found {len(files)} samples")

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(labels)

# ==============================
# FILTER CLASSES WITH <2 SAMPLES
# ==============================
label_counts = Counter(y_encoded)
filtered_indices = [i for i, y in enumerate(y_encoded) if label_counts[y] >= 2]

files = [files[i] for i in filtered_indices]
y_encoded = y_encoded[filtered_indices]

# Re-encode after filtering
le = LabelEncoder()
y_encoded = le.fit_transform(y_encoded)
num_classes = len(le.classes_)
print(f"Number of gloss classes after filtering: {num_classes}")

# ==============================
# TRAIN/VAL/TEST SPLIT
# ==============================
files_train, files_tmp, y_train, y_tmp = train_test_split(
    files, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)
files_val, files_test, y_val, y_test = train_test_split(
    files_tmp, y_tmp, test_size=0.5, stratify=y_tmp, random_state=42
)

# ==============================
# CLASS WEIGHTS & LOSS
# ==============================
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.arange(num_classes),
    y=y_train
)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(DEVICE)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

# ==============================
# DATASET & DATALOADERS
# ==============================
train_dataset = LandmarkDataset(files_train, y_train)
val_dataset = LandmarkDataset(files_val, y_val)
test_dataset = LandmarkDataset(files_test, y_test)

# WeightedRandomSampler for class balancing
sample_weights = np.array([class_weights[y] for y in y_train])
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# ==============================
# IMPROVED 1D CNN
# ==============================
class CNN1D(nn.Module):
    def __init__(self, input_features, num_classes):
        super(CNN1D, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv1d(input_features, 256, kernel_size=3, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Conv1d(256, 128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.05),
            
            nn.Conv1d(128, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1)
        )
        self.fc = nn.Linear(64, num_classes)
    
    def forward(self, x):
        x = self.conv_block(x)
        x = x.squeeze(-1)
        x = self.fc(x)
        return x

model = CNN1D(FEATURE_DIM, num_classes).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=3
)
print(model)

# ==============================
# TRAINING LOOP WITH EARLY STOPPING
# ==============================
def train_model(model, train_loader, val_loader, epochs, patience):
    best_val_acc = 0.0
    counter = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]"):
            x, y = x.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * x.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

        train_loss = running_loss / total
        train_acc = correct / total

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for x_val, y_val in val_loader:
                x_val, y_val = x_val.to(DEVICE), y_val.to(DEVICE)
                outputs = model(x_val)
                preds = outputs.argmax(dim=1)
                val_correct += (preds == y_val).sum().item()
                val_total += y_val.size(0)
        val_acc = val_correct / val_total

        # Step scheduler and print if LR changed
        old_lr = optimizer.param_groups[0]['lr']
        scheduler.step(val_acc)
        new_lr = optimizer.param_groups[0]['lr']
        if new_lr != old_lr:
            print(f"  🔹 LR reduced from {old_lr:.6f} to {new_lr:.6f}")

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            counter = 0
            torch.save(model.state_dict(), "CNN_with_SW_hands_only_best_model.pth")
            print("  🔹 Saved best model")
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping triggered!")
                break

    print(f"Training finished. Best val acc: {best_val_acc:.4f}")

# ==============================
# EVALUATION
# ==============================
def evaluate_model(model, test_loader, label_encoder):
    # Load best model
    model.load_state_dict(torch.load("CNN_with_SW_hands_only_best_model.pth"))
    model.eval()
    
    all_preds, all_labels = [], []

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            outputs = model(x)
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())
    
    # Compute accuracy
    acc = np.mean(np.array(all_preds) == np.array(all_labels))
    print(f"Test Accuracy: {acc:.4f}\n")

    # Convert integer labels back to string labels
    all_preds_str = label_encoder.inverse_transform(all_preds)
    all_labels_str = label_encoder.inverse_transform(all_labels)

    # Classification report
    print("Classification Report:")
    print(classification_report(all_labels_str, all_preds_str))

    # Confusion matrix
    print("\nConfusion Matrix:")
    print(confusion_matrix(all_labels_str, all_preds_str))

# ==============================
# RUN
# ==============================
train_model(model, train_loader, val_loader, EPOCHS, PATIENCE)
evaluate_model(model, test_loader, le)


Using device: cpu
Found 5845 samples
Number of gloss classes after filtering: 146
CNN1D(
  (conv_block): Sequential(
    (0): Conv1d(438, 256, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.1, inplace=False)
    (4): Conv1d(256, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.05, inplace=False)
    (8): Conv1d(128, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): AdaptiveMaxPool1d(output_size=1)
  )
  (fc): Linear(in_features=64, out_features=146, bias=True)
)


Epoch 1/60 [Train]: 100%|████████████████████████████████████████████████████████████| 293/293 [00:26<00:00, 11.20it/s]


Epoch 1/60 | Train Loss: 4.6705 | Train Acc: 0.0351 | Val Acc: 0.0360
  🔹 Saved best model


Epoch 2/60 [Train]: 100%|████████████████████████████████████████████████████████████| 293/293 [00:26<00:00, 11.02it/s]


Epoch 2/60 | Train Loss: 3.8527 | Train Acc: 0.0877 | Val Acc: 0.1010
  🔹 Saved best model


...

Epoch 49/60 | Train Loss: 0.0459 | Train Acc: 0.9840 | Val Acc: 0.8014


Epoch 50/60 [Train]: 100%|███████████████████████████████████████████████████████████| 293/293 [00:21<00:00, 13.94it/s]


Epoch 50/60 | Train Loss: 0.0424 | Train Acc: 0.9874 | Val Acc: 0.8048


Epoch 51/60 [Train]: 100%|███████████████████████████████████████████████████████████| 293/293 [00:20<00:00, 14.02it/s]


Epoch 51/60 | Train Loss: 0.0417 | Train Acc: 0.9861 | Val Acc: 0.8014
Early stopping triggered!
Training finished. Best val acc: 0.8151
Test Accuracy: 0.8239

Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.57      0.62         7
           1       0.67      0.67      0.67         3
           2       1.00      1.00      1.00         3
           3       0.67      0.67      0.67         3
           4       0.75      0.75      0.75         4
           5       1.00      1.00      1.00         7
           6       0.67      0.67      0.67         3
           7       0.67      0.67      0.67         3
           8       1.00      1.00      1.00         3
           9       0.75      1.00      0.86         3
          10       0.75      1.00      0.86         3
          11       0.60      1.00      0.75         3
          12       1.00      0.67      0.80         3
          13       1.00      1.00      1.00         3
      